In [ ]:
#install.packages(c("httr", "jsonlite"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [1]:
library(httr)
library(jsonlite)
library(data.table)
library(dtplyr)

In [6]:
cms_data_importer=function(url,data_size){

  require(data.table)
  require(dplyr)
  require(httr)
  require(jsonlite)
  require(tidyverse)
  
  stats=GET(paste(url,"/stats",sep = ""))

  row_count=content(stats,as="parsed")$total_rows

  offset=0
  data=data.table()

  if(data_size=="all"){
    while(offset<row_count){
      new_get=GET(paste(url,"?size=5000&offset=",offset,sep = ""))
      new_data=fromJSON(rawToChar(new_get$content))%>%as.data.table()
      data=rbind(data,new_data)
      offset=offset+5000
    }
  }
  else{
    while(offset<data_size){
      new_get=GET(paste(url,"?size=5000&offset=",offset,sep = ""))
      new_data=fromJSON(rawToChar(new_get$content))%>%as.data.table()
      data=rbind(data,new_data)
      offset=offset+5000
    }
  }
  
  return(data)

}

In [21]:
physician_data=cms_data_importer(
  url = 
  "https://data.cms.gov/data-api/v1/dataset/14cd68c2-9433-4069-90fa-d213ef709661/data",
  data_size=1000
)

In [22]:
colnames(physician_data)

[1] "Rndrng_NPI"                    "Rndrng_Prvdr_Last_Org_Name"   
 [3] "Rndrng_Prvdr_First_Name"       "Rndrng_Prvdr_MI"              
 [5] "Rndrng_Prvdr_Crdntls"          "Rndrng_Prvdr_Gndr"            
 [7] "Rndrng_Prvdr_Ent_Cd"           "Rndrng_Prvdr_St1"             
 [9] "Rndrng_Prvdr_St2"              "Rndrng_Prvdr_City"            
[11] "Rndrng_Prvdr_State_Abrvtn"     "Rndrng_Prvdr_State_FIPS"      
[13] "Rndrng_Prvdr_Zip5"             "Rndrng_Prvdr_RUCA"            
[15] "Rndrng_Prvdr_RUCA_Desc"        "Rndrng_Prvdr_Cntry"           
[17] "Rndrng_Prvdr_Type"             "Rndrng_Prvdr_Mdcr_Prtcptg_Ind"
[19] "HCPCS_Cd"                      "HCPCS_Desc"                   
[21] "HCPCS_Drug_Ind"                "Place_Of_Srvc"                
[23] "Tot_Benes"                     "Tot_Srvcs"                    
[25] "Tot_Bene_Day_Srvcs"            "Avg_Sbmtd_Chrg"               
[27] "Avg_Mdcr_Alowd_Amt"            "Avg_Mdcr_Pymt_Amt"            
[29] "Avg_Mdcr_Stdzd_Amt"

In [23]:
physician_data=physician_data[,.(Rndrng_NPI,Rndrng_Prvdr_Type,HCPCS_Cd,Tot_Srvcs)]

In [24]:
head(physician_data, 20)
dim(physician_data)

Rndrng_NPI,Rndrng_Prvdr_Type,HCPCS_Cd,Tot_Srvcs
<chr>,<chr>,<chr>,<chr>
1003000126,Internal Medicine,99217,23
1003000126,Internal Medicine,99218,16
1003000126,Internal Medicine,99220,16
1003000126,Internal Medicine,99221,12
1003000126,Internal Medicine,99223,52
1003000126,Internal Medicine,99232,134
1003000126,Internal Medicine,99233,326
1003000126,Internal Medicine,99238,13
1003000126,Internal Medicine,99239,146


[1] 5000    4

In [25]:
physician_data[,Tot_Srvcs:=as.numeric(Tot_Srvcs)]

In [26]:
physician_data_wide=dcast(physician_data, Rndrng_NPI+Rndrng_Prvdr_Type ~ HCPCS_Cd,
 value.var = 'Tot_Srvcs', fun.aggregate = sum)%>%
 as.data.table()

In [27]:
head(physician_data_wide)
dim(physician_data_wide)

Rndrng_NPI,Rndrng_Prvdr_Type,0042T,0055T,0191T,0509T,10005,10060,104,11042,⋯,J9217,J9250,Q0091,Q4186,Q9956,Q9957,Q9965,Q9966,Q9967,U0002
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1003000126,Internal Medicine,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
1003000134,Pathology,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
1003000142,Anesthesiology,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
1003000423,Obstetrics & Gynecology,0,0,0,0,0,0,0,0,⋯,0,0,15,0,0,0,0,0,0,0
1003000480,General Surgery,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
1003000522,Family Practice,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] 482 941

In [28]:
physician_data_wide[Rndrng_NPI==1003000126,"99217"]

99217
<dbl>
23


In [30]:
distance_finder=function(x,y){
  sqrt(sum((x-y)^2, na.rm = T))
}

In [31]:
x=c(1,2,3,4,5)
y=c(2,3,4,5,6)
distance_finder(x,y)

[1] 2.236068

In [32]:
2.23606797749979^2

[1] 5

In [33]:
install.packages("caTools")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’




In [34]:
library(caTools)
set.seed=123
sample=sample.split(SplitRatio = 0.8 , physician_data_wide$Rndrng_NPI)
test_set=subset(physician_data_wide,sample==F)%>%as.data.frame
training_set=subset(physician_data_wide,sample==T)%>%as.data.frame

In [35]:
head(training_set)

,Rndrng_NPI,Rndrng_Prvdr_Type,0042T,0055T,0191T,0509T,10005,10060,104,11042,⋯,J9217,J9250,Q0091,Q4186,Q9956,Q9957,Q9965,Q9966,Q9967,U0002
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1003000126,Internal Medicine,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,1003000134,Pathology,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,1003000423,Obstetrics & Gynecology,0,0,0,0,0,0,0,0,⋯,0,0,15,0,0,0,0,0,0,0
4,1003000480,General Surgery,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,1003000522,Family Practice,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,1003000530,Internal Medicine,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [38]:
install.packages(c("foreach","doMC"))

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘iterators’




In [39]:
knn=function(k=1,train_data,test_data,name_var,outcome_var,mode="prediction",ncores=2){
  
  require(data.table)
  require(dtplyr)
  require(tidyverse)
  library(foreach)
  library(doMC)
  registerDoMC(cores=ncores)

  distance_finder=function(x,y){
    sqrt(sum((x-y)^2,na.rm = T))
  }
  
  

  result=
  foreach(a = 1:nrow(test_data), .combine=rbind) %:%
    foreach(b = 1:nrow(train_data) , .combine=rbind) %dopar% {
      data.frame(distance_finder(test_data[a,!names(test_data) %in% c(name_var,outcome_var)],
                      train_data[b,!names(train_data) %in% c(name_var,outcome_var)]),
        train_data[b,c(name_var,outcome_var)],
        test_data[a,c(name_var,outcome_var)])
    }


  colnames(result)=
  c("Distance",
    paste("training",name_var,sep="_"),
    paste("training",outcome_var,sep="_"),
    paste("test",name_var,sep="_"),
    paste("test",outcome_var,sep="_"))

  result=result%>%
  group_by(eval(parse(text=paste("test",outcome_var,sep="_"))))%>%
  arrange(Distance,.by_group = T)%>%
  as.data.table()

  Mode = function(x) {
    uniques = unique(x)
    uniques[which.max(tabulate(match(x, uniques)))]
    }

  prediction=result[,prediction:= Mode(head(eval(parse(text=paste("training",outcome_var,sep="_"))),k))
                    ,by=eval(parse(text=paste("test",outcome_var,sep="_")))]

  
  result=result[,head(.SD,k),by=eval(parse(text=paste("test",name_var,sep="_")))]
  
  colnames(result)[1]=paste("test",name_var,sep="_")
  colnames(result)[5]="expected"
  result=result[,-6]
  
  if(mode=="diagnosis"){
    result=result
  }
  if(mode=="prediction"){
    result=result[,head(.SD,1),by=eval(parse(text=paste("test",name_var,sep="_")))]
    result=result[,.(expected,prediction)]
  }


  return(result)

}

In [40]:
a=knn(k=2,
      train_data=training_set[1:10,],
      test_data = test_set[1:100,],
      ncores = 2,
      outcome_var = "Rndrng_Prvdr_Type",
      name_var = "Rndrng_NPI",
      mode="diagnosis"
     )


Attaching package: ‘foreach’


The following objects are masked from ‘package:purrr’:

    accumulate, when


Loading required package: iterators

Loading required package: parallel



In [ ]:
a

test_Rndrng_NPI,Distance,training_Rndrng_NPI,training_Rndrng_Prvdr_Type,expected,prediction
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>
1003021114,2.828427,1003000480,General Surgery,Allergy/ Immunology,General Surgery
1003021114,2.828427,1003000639,Cardiac Surgery,Allergy/ Immunology,General Surgery
1003015355,1.732051,1003000639,Cardiac Surgery,Ambulance Service Provider,Cardiac Surgery
1003015355,1.732051,1003000704,Anesthesiology,Ambulance Service Provider,Cardiac Surgery
1003017351,3.741657,1003000639,Cardiac Surgery,Ambulatory Surgical Center,Cardiac Surgery
1003017351,3.741657,1003000704,Anesthesiology,Ambulatory Surgical Center,Cardiac Surgery
1003007360,1.414214,1003000639,Cardiac Surgery,Anesthesiology,Cardiac Surgery
1003007360,1.414214,1003000704,Anesthesiology,Anesthesiology,Cardiac Surgery
1003011966,1.414214,1003000704,Anesthesiology,Anesthesiology,Cardiac Surgery
